### Make sure to install ChromeDriver here (May need to update which chromedriver is saved in current working python directory):
https://sites.google.com/a/chromium.org/chromedriver/downloads

https://towardsdatascience.com/web-scraping-using-selenium-python-8a60f4cf40ab

In [1]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup


from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
from datetime import datetime
#from datetime import date
#import datetime


time_of_search = datetime.now().date()

In [2]:

from sqlalchemy import create_engine
from sqlalchemy import(Table, Column, String, Integer, Boolean)
from sqlalchemy import MetaData, Table

engine = create_engine('sqlite:///LME_nickel.sqlite')

connection = engine.connect()

print(engine.table_names())


['daily_nickel_prices']


In [3]:
import datetime

def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    beautifulsoup object.
    """
    try:
        #with closing(get(url, stream=True)) as resp:
        chrome_options = webdriver.ChromeOptions()
        driver = webdriver.Chrome("chromedriver")
        driver.implicitly_wait(20)
        driver.get(url)
        soup = BeautifulSoup(driver.page_source, 'lxml')
        if type(driver) == webdriver.chrome.webdriver.WebDriver:
            return soup, driver
        else:
            return None

    except Exception:
        # MAIN ERROR? 
        #log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        print('Error retrieving contents at {}'.format(url))
        return None

In [6]:
def run_scrape():
    #url = f'https://www.lme.com/en-GB/Metals/Non-ferrous/Nickel#tabIndex=0'
    url = 'https://www.lme.com/en-GB/Metals/Non-ferrous/Nickel#tabIndex=2'
    soup, driver = simple_get(url)

    soup = BeautifulSoup(driver.page_source, 'lxml')

    #h2 = soup.find('div', id="LME NICKEL OFFICIAL PRICES, US$ PER TONNE").h2

    html = soup.find_all('div', attrs={'id': 'module-63'})
    text_list = html[0].getText().strip().split()


    for i,item in enumerate(text_list):
        if item.lower() == 'cash':
            nickel_price = text_list[i+1]
    nickel_price2 = pd.DataFrame(pd.Series(nickel_price))
    nickel_price2.index = [time_of_search]
    nickel_price2.columns = ['0']
    nickel_price2
    
    return nickel_price2
'''

Data structure of forecasting project - 
Use SQL Lite and CSV format to have human review data scrape incase of errors


'''
nickel_price = run_scrape()

nickel_price


,0
2019-12-09,13415.00


In [5]:
nickel_price

,0
2019-12-09,13415.00


In [7]:
url = 'https://www.lme.com/en-GB/Metals/Non-ferrous/Nickel#tabIndex=2'
soup, driver = simple_get(url)

soup = BeautifulSoup(driver.page_source, 'lxml')

#h2 = soup.find('div', id="LME NICKEL OFFICIAL PRICES, US$ PER TONNE").h2

html = soup.find_all('div', attrs={'id': 'module-63'})
text_list = html[0].getText().strip().split()


for i,item in enumerate(text_list):
    if item.lower() == 'cash':
        nickel_price = text_list[i+1]
nickel_price2 = pd.DataFrame(pd.Series(nickel_price))
nickel_price2.index = [time_of_search]
nickel_price2.columns = ['0']
nickel_price2



,0
2019-12-09,13415.00


In [185]:
df = pd.read_sql_query('SELECT * from daily_nickel_prices;', connection)
df.index = pd.to_datetime(df['index'])
df = df.iloc[:,1:]
df = df.append(nickel_price2)
#nickel_price.to_sql("daily_nickel_prices", con=engine, if_exists='replace', index = True)


In [2]:
#df = pd.concat([df, nickel_price2])
connection.close()

In [8]:
text_list

['Contract',
 'Bid',
 '(US$',
 '/',
 'Tonne)',
 'Offer',
 '(US$',
 '/',
 'Tonne)*',
 'Cash',
 '13415.00',
 '13420.00',
 '3-months',
 '13430.00',
 '13480.00',
 'Dec',
 '20',
 '13560.00',
 '13610.00',
 'Dec',
 '21',
 '13765.00',
 '13815.00']

In [7]:
import schedule
import time

In [14]:
schedule.every(2).minutes.do(run_scrape)

while True:
    schedule.run_pending()
    time.sleep(1)

13240.00
13240.00
13240.00


KeyboardInterrupt: 

### Testing